In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense

2024-09-18 09:33:25.975793: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/macfarqu/Desktop/My_Projects/PCGML/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


PRE_PROCESS

In [11]:
df = pd.read_csv("cards.csv", usecols=['name', 'type', 'atk', 'def', 'level', 'race'])
df['atk'] = df['atk'].fillna(0)
df['def'] = df['def'].fillna(0)
df['level'] = df['level'].fillna(0)

DELIM_TOKEN = "DELIM"
df['text'] = df.apply(lambda row: f"{DELIM_TOKEN} {row['name']} {DELIM_TOKEN} {row['type']} {DELIM_TOKEN} ATK {int(row['atk'])} {DELIM_TOKEN} DEF {int(row['def'])} {DELIM_TOKEN} Level {int(row['level'])} {DELIM_TOKEN} {row['race']} {DELIM_TOKEN}", axis=1)

print (len(df))

entries = df['text']
print(entries[:10])

13281
0    DELIM "A" Cell Breeding Device DELIM Spell Car...
1    DELIM "A" Cell Incubator DELIM Spell Card DELI...
2    DELIM "A" Cell Recombination Device DELIM Spel...
3    DELIM "A" Cell Scatter Burst DELIM Spell Card ...
4    DELIM "Infernoble Arms - Almace" DELIM Spell C...
5    DELIM "Infernoble Arms - Durendal" DELIM Spell...
6    DELIM "Infernoble Arms - Hauteclere" DELIM Spe...
7    DELIM "Infernoble Arms - Joyeuse" DELIM Spell ...
8    DELIM 1st Movement Solo DELIM Spell Card DELIM...
9    DELIM 3-Hump Lacooda DELIM Effect Monster DELI...
Name: text, dtype: object


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(entries)
total_words = len(tokenizer.word_index) + 1

print(total_words) # VOCAB SIZE
print(tokenizer.to_json())

10173
{"class_name": "Tokenizer", "config": {"num_words": null, "filters": "!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n", "lower": true, "split": " ", "char_level": false, "oov_token": null, "document_count": 13281, "word_counts": "{\"delim\": 92967, \"a\": 109, \"cell\": 11, \"breeding\": 2, \"device\": 8, \"spell\": 2636, \"card\": 4689, \"atk\": 13281, \"0\": 16517, \"def\": 13281, \"level\": 13296, \"continuous\": 986, \"incubator\": 1, \"recombination\": 1, \"quick\": 500, \"play\": 499, \"scatter\": 2, \"burst\": 25, \"infernoble\": 15, \"arms\": 26, \"almace\": 1, \"equip\": 274, \"durendal\": 2, \"hauteclere\": 1, \"joyeuse\": 1, \"1st\": 1, \"movement\": 1, \"solo\": 3, \"normal\": 2948, \"3\": 1215, \"hump\": 1, \"lacooda\": 2, \"effect\": 5379, \"monster\": 8545, \"500\": 579, \"1500\": 885, \"beast\": 1187, \"30\": 2, \"000\": 1, \"year\": 1, \"white\": 76, \"turtle\": 23, \"1250\": 13, \"2100\": 362, \"5\": 644, \"aqua\": 294, \"4\": 2478, \"starred\": 1, \"ladybug\": 6, \"of\":

In [13]:
input_sequences = []
for entry in entries:
  token_list = tokenizer.texts_to_sequences([entry])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

print(len(input_sequences))
print(input_sequences[:10])

max_sequence_len = max([len(x) for x in input_sequences])
print(max_sequence_len)

238714
[[1, 101], [1, 101, 677], [1, 101, 677, 2246], [1, 101, 677, 2246, 882], [1, 101, 677, 2246, 882, 1], [1, 101, 677, 2246, 882, 1, 10], [1, 101, 677, 2246, 882, 1, 10, 8], [1, 101, 677, 2246, 882, 1, 10, 8, 1], [1, 101, 677, 2246, 882, 1, 10, 8, 1, 4], [1, 101, 677, 2246, 882, 1, 10, 8, 1, 4, 2]]
27


In [14]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = to_categorical(label, num_classes=total_words)

LSTM Model

In [15]:
model = Sequential()

model.add(Embedding(total_words, 16, input_shape=(max_sequence_len,)))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

/Users/macfarqu/Desktop/My_Projects/PCGML/venv/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 27, 16)         │       162,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │        46,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10173)          │     1,027,473 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,237,041 (4.72 MB)

 Trainable params: 1,237,041 (4.72 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(predictors, label, epochs=50, batch_size=64)


Epoch 1/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 112s 30ms/step - loss: 1.8943
Epoch 2/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 125s 33ms/step - loss: 1.7316
Epoch 3/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 115s 31ms/step - loss: 1.6146
Epoch 4/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 114s 31ms/step - loss: 1.5445
Epoch 5/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 121s 32ms/step - loss: 1.4721
Epoch 6/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 121s 33ms/step - loss: 1.4262
Epoch 7/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 125s 33ms/step - loss: 1.3681
Epoch 8/50
3730/3730 ━━━━━━━━━━━━━━━━━━━━ 127s 34ms/step - loss: 1.3290
Epoch 9/50
 410/3730 ━━━━━━━━━━━━━━━━━━━━ 2:04 38ms/step - loss: 1.2598

KeyboardInterrupt: 

GENERATE

In [22]:
text = "delim conjuring"

delims_seen = 2
while delims_seen < 7:
  token_list = tokenizer.texts_to_sequences([text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model.predict(token_list, verbose=0)[0]
  predicted_index = np.random.choice(len(predicted), p=predicted)

  out_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted_index:
      out_word = word
      break
  if out_word == "delim":
    delims_seen += 1
  text += " " + out_word

cleaned_text = text.replace("delim", "|")
print(cleaned_text)

| conjuring blustering magician | effect monster | atk 1500 | def 2100 | level 6 |
